In [ ]:
import gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from nes_py.wrappers import JoypadSpace
import os 

# Função para criar o ambiente personalizado
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v1')
    env = JoypadSpace(env, RIGHT_ONLY)
    env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
    env = CustomRescaleObservation(env, low=0, high=255)
    env = gym.wrappers.FrameStack(env, num_stack=6)
    return env

# Classe personalizada para redimensionar as observações
class CustomRescaleObservation(gym.ObservationWrapper):
    def __init__(self, env, low, high):
        super(CustomRescaleObservation, self).__init__(env)
        self.low = low
        self.high = high

    def observation(self, observation):
        return observation * (self.high - self.low) + self.low

# Callback personalizado para recompensas e penalidades
class CustomCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.x_pos = None

    def _on_step(self):
        # Obtem informações sobre o ambiente após a etapa
        info = self.locals.get("info", {})

        # Recompensa o agente em 1 ponto se ele uma moeda
        if "coins" in info and info["coins"] > 0:
            self.rewards += 1
            self.logger.record("coin_reward", self.rewards)

        # Penaliza o agente em 15 pontos se morrer
        if "life" in info and info["life"] < 2:
            self.penalties += 15
            self.logger.record("death_penalty", self.penalties)

        # Verifica se o agente está parado na mesma posição
        if "x_pos" in info and info["x_pos"] == self.x_pos:
            # Penaliza o agente em 0.3 pontos se ficar em uma mesma posição
            self.penalties += 0.3
            self.logger.record("stagnation_penalty", self.penalties)
            # Verifiqua a altura do agente 
            if "y_pos" in info and info["y_pos"] > 200:
                # Recompensa o agente com 10 pontos ao executar um pulo com uma altura maior que 200
                self.rewards += 10
                self.logger.record("height_reward", 10)
            
            if "y_pos" in info and info["y_pos"] < 150:
                # Penaliza o agente com 0.3 pontos ao executar um pulo com uma altura menor que 150
                self.penalties += 0.3
                self.logger.record("stagnation_penalty", self.penalties)

        # Salva a posição atual para a próxima iteração
        self.x_pos = info.get("x_pos", None)
        self.y_pos = info.get("y_pos", None)

        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'melhor_modelo_mario_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

# Cria o ambiente do jogo
env = make_env()

# Resgata um modelo salvo
model = PPO.load('./mario_model_v4/melhor_modelo_mario_200000')

# Treina o modelo com o callback personalizado
custom_callback = CustomCallback()

state = env.reset()

done = True
for step in range(100000): 
     
    if done:     
        env.reset()

    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    print("Altura do agente:", info.get("y_pos", None))
    os.system('clear')

    env.render()
env.close()
